In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import sys
import time
import tqdm
import pickle
import feather
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
text = pd.read_csv('../input/chinese_news.csv', usecols=['tag', 'headline', 'content'])
dates = pd.read_csv('../input/chinese_news.csv', usecols=['date'])
text.shape

In [ ]:
text.head()

In [ ]:
dates['datetime'] = dates['date'].apply(lambda x: pd.to_datetime(x))
dates['year'] = dates['datetime'].dt.year
dates['month'] = dates['datetime'].dt.month
dates['dow'] = dates['datetime'].dt.dayofweek

In [ ]:
plt.figure(figsize=[8, 4])
sns.countplot(x='year', data=dates)
plt.title('News count by day of week')
plt.ylabel('News count');
plt.xlabel('Year(where year 2018 is up to 10/09)');

In [ ]:
plt.figure(figsize=[10, 5])
sns.countplot(x='dow', data=dates)
plt.title('News count by day of week')
plt.ylabel('News count');
plt.xlabel('Day of week');

In [ ]:
plt.figure(figsize=[12, 6])
sns.countplot(x='month', data=dates)
plt.title('News count by month')
plt.ylabel('News count');
plt.xlabel('Month');

In [ ]:
!wget https://github.com/adobe-fonts/source-han-sans/raw/release/SubsetOTF/SourceHanSansCN.zip
!unzip -j "SourceHanSansCN.zip" "SourceHanSansCN/SourceHanSansCN-Regular.otf" -d "."
!rm SourceHanSansCN.zip
!ls

In [ ]:
import matplotlib.font_manager as fm
font_path = './SourceHanSansCN-Regular.otf'
prop = fm.FontProperties(fname=font_path)

In [ ]:
plt.figure(figsize=[8, 4])
ax = sns.countplot(x='tag', data=text)
plt.title('News count by tag')
plt.ylabel('News count')
plt.xlabel('Tag')
ax.set_xticklabels(ax.get_xticklabels(), fontproperties=prop);

In [ ]:
!pip install jieba

In [ ]:
import jieba

def jieba_cut(x, sep=' '):
    return sep.join(jieba.cut(x, cut_all=False))

print('raw', text['headline'][0])
print('cut', jieba_cut(text['headline'][0], ', '))

In [ ]:
from joblib import Parallel, delayed

In [ ]:
%%time
text['headline_cut'] = Parallel(n_jobs=4)(
    delayed(jieba_cut)(x) for x in tqdm.tqdm_notebook(text['headline'].values)
)

In [ ]:
%%time
text['content_cut'] = Parallel(n_jobs=4)(
    delayed(jieba_cut)(x) for x in tqdm.tqdm_notebook(text['content'].fillna('').values)
)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator

def get_wc(
    text_li, 
    background_color='white',
    font_path=font_path,
    width=640,
    height=480,
    max_font_size=64,
    mask=None,
    margin=1,
):
    return WordCloud(
        background_color=background_color,
        font_path=font_path,
        width=width,
        height=height,
        max_font_size=max_font_size,
        mask=mask,
        margin=margin,
    ).generate(" ".join(text_li))

In [ ]:
text_li = text['headline_cut'].values.tolist()
wc = get_wc(text_li)
plt.figure(figsize=[12, 8])
plt.imshow(wc)
plt.title('All headlines')
plt.axis('off');

In [ ]:
text_li = text['content_cut'].values.tolist()
wc = get_wc(text_li)
plt.figure(figsize=[12, 8])
plt.imshow(wc)
plt.title('All contents')
plt.axis('off');

In [ ]:
plt.figure(figsize=[8*3, 6])
tags = text['tag'].unique()
for i,op in enumerate([('tag', tags[0]), ('tag', tags[1]), ('tag', tags[2])]):
    plt.subplot(1, 3, i+1)
    text_li = text.loc[text[op[0]]==op[1], 'headline_cut'].values.tolist()
    wc = get_wc(text_li)
    plt.imshow(wc)
    plt.title(f'Headlines of {op[0]} {op[1]}', fontproperties=prop)
    plt.axis('off');
plt.tight_layout();

In [ ]:
plt.figure(figsize=[8*3, 6])
for i,op in enumerate([('year', 2016), ('year', 2017), ('year', 2018)]):
    plt.subplot(1, 3, i+1)
    text_li = text.loc[dates[op[0]]==op[1], 'headline_cut'].values.tolist()
    wc = get_wc(text_li)
    plt.imshow(wc)
    plt.title(f'Headlines of {op[0]} {op[1]}')
    plt.axis('off');
plt.tight_layout();

In [ ]:
plt.figure(figsize=[8*3, 6*4])
for i,m in enumerate(range(12)):
    op = ('month', m+1)
    plt.subplot(4, 3, i+1)
    text_li = text.loc[dates[op[0]]==op[1], 'headline_cut'].values.tolist()
    wc = get_wc(text_li)
    plt.imshow(wc)
    plt.title(f'Headlines of {op[0]} {op[1]}')
    plt.axis('off');
plt.tight_layout();

In [ ]:
!wget -O 'mask.jpg' -q http://p3.pstatp.com/large/37cb00016104f26b1608 

In [ ]:
from PIL import Image
mask = np.array(Image.open('mask.jpg'))[0:320, 156:484, :]
maskb = mask.copy()
maskb[maskb<90]=0
maskb[maskb>=90]=255
maskb = 255 - maskb
mask = 0.8 * mask
mask[maskb==255] = 255

text_li = text['headline_cut'].values.tolist()
wc = get_wc(text_li, mask=mask)
plt.figure(figsize=[12, 12])
plt.imshow(wc.recolor(color_func=ImageColorGenerator(mask)))
plt.imshow(np.array(Image.open('mask.jpg'))[0:320, 156:484, :], alpha=0.5)
plt.title('JING')
plt.axis('off');